In [1]:
import argparse
import os
import torch
import numpy as np
import cv2
!pip install hdf5storage
import hdf5storage as hdf5
import sys





     |████████████████████████████████| 53 kB 759 kB/s 


In [2]:
sys.path.insert(0,"/content/drive/My Drive/Hyperspectral")

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
os.chdir("/content/gdrive/MyDrive/Hyperspectral/")
print(os.getcwd())
import utils_hp as utils
import dataset

Mounted at /content/gdrive
/content/gdrive/MyDrive/Hyperspectral


# New Section

In [3]:
#if __name__ == "__main__":
    # ----------------------------------------
    #        Initialize the parameters
    # ----------------------------------------
%tb
parser = argparse.ArgumentParser()
# Pre-train, saving, and loading parameters
parser.add_argument('--pre_train', type = bool, default = True, help = 'pre_train or not')
parser.add_argument('--test_batch_size', type = int, default = 1, help = 'size of the testing batches for single GPU')
parser.add_argument('--num_workers', type = int, default = 2, help = 'number of cpu threads to use during batch generation')
parser.add_argument('--val_path', type = str, default = './test', help = 'saving path that is a folder')
parser.add_argument('--task_name', type = str, default = 'track2', help = 'task name for loading networks, saving, and log')
# Network initialization parameters
parser.add_argument('--pad', type = str, default = 'reflect', help = 'pad type of networks')
parser.add_argument('--activ', type = str, default = 'lrelu', help = 'activation type of networks')
parser.add_argument('--norm', type = str, default = 'none', help = 'normalization type of networks')
parser.add_argument('--in_channels', type = int, default = 3, help = 'input channels for generator')
parser.add_argument('--out_channels', type = int, default = 31, help = 'output channels for generator')
parser.add_argument('--start_channels', type = int, default = 64, help = 'start channels for generator')
parser.add_argument('--init_type', type = str, default = 'xavier', help = 'initialization type of generator')
parser.add_argument('--init_gain', type = float, default = 0.02, help = 'initialization gain of generator')
# Dataset parameters
parser.add_argument('--baseroot', type = str, default = './NTIRE2020_Test_Clean', help = 'baseroot')
# NTIRE2020_Validation_Clean    NTIRE2020_Validation_RealWorld
opt = parser.parse_args(args=[])

# ----------------------------------------
#                   Test
# ----------------------------------------
load_net_name_list = []
load_net_name_list.append('./track1/code1_320_G_epoch9000_bs4.pth')
load_net_name_list.append('./track1/code1_384_G_epoch6000_bs4.pth')
load_net_name_list.append('./track1/code1_bs2_G_epoch8000_bs2.pth')
load_net_name_list.append('./track1/code1_bs4_G_epoch9000_bs4.pth')
load_net_name_list.append('./track1/code1_first_G_epoch10000_bs8.pth')
load_net_name_list.append('./track1/code1_G_epoch9000_bs8.pth')
load_net_name_list.append('./track1/code1_second_G_epoch8000_bs8.pth')
load_net_name_list.append('./track1/code2_G_epoch7000_bs8.pth')

for i, name in enumerate(load_net_name_list):
    if i < 7:
        # Initialize
        generator = utils.create_generator_val1(opt, name).cuda()
        test_dataset = dataset.HS_multiscale_ValDSet(opt)
        print(test_dataset.imglist)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = opt.test_batch_size, shuffle = False, num_workers = opt.num_workers, pin_memory = True)
        sample_folder = os.path.join(opt.val_path, opt.task_name, str(i))
        utils.check_path(sample_folder)
        print('Network name: %s; The %d-th iteration' % (name, i))

        # forward
        for j, (img1, img2, path) in enumerate(test_loader):
            # To device
            img1 = img1.cuda()
            img2 = img2.cuda()
            path = path[0]
            print(opt.task_name, i, path)

            # Forward propagation
            with torch.no_grad():
                out1 = generator(img1) # [0:480, 0:512, :], [1, 31, 480, 512]
                out2 = generator(img2) # [2:482, 0:512, :], [1, 31, 480, 512]
            out = torch.cat((out1, out2[:, :, 478:, :]), 2)

            # Save
            out = out.clone().data.permute(0, 2, 3, 1).cpu().numpy()[0, :, :, :].astype(np.float64)
            save_img_name = path[:12] + '.mat'
            save_img_path = os.path.join(sample_folder, save_img_name)
            hdf5.write(data = out, path = 'cube', filename = save_img_path, matlab_compatible = True)
            #out[:,:,0:31]
            


    if i == 7:
        # Initialize
        generator = utils.create_generator_val2(opt, name).cuda()
        test_dataset = dataset.HS_multiscale_ValDSet(opt)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = opt.test_batch_size, shuffle = False, num_workers = opt.num_workers, pin_memory = True)
        sample_folder = os.path.join(opt.val_path, opt.task_name, str(i))
        utils.check_path(sample_folder)
        print('Network name: %s; The %d-th iteration' % (name, i))

        # forward
        for j, (img1, img2, path) in enumerate(test_loader):
            # To device
            img1 = img1.cuda()
            #print(img1)
            img2 = img2.cuda()
            #print(img2)
            path = path[0]
            print(opt.task_name, i, path)

            # Forward propagation
            with torch.no_grad():
                out1 = generator(img1) # [0:480, 0:512, :], [1, 31, 480, 512]
                out2 = generator(img2) # [2:482, 0:512, :], [1, 31, 480, 512]
            out = torch.cat((out1, out2[:, :, 478:, :]), 2)
            #print(type(out))

            # Save
            out = out.clone().data.permute(0, 2, 3, 1).cpu().numpy()[0, :, :, :].astype(np.float64)
            save_img_name = path[:12] + '.mat' #.npy for array
            save_img_path = os.path.join(sample_folder, save_img_name)
            print(save_img_path)
            hdf5.write(data = out, path = 'cube', filename = save_img_path, matlab_compatible = True)


No traceback available to show.


Generator is loaded!
['t125.jpeg', 't126.jpeg', 't127.jpeg', 't128.jpeg', 't129.jpeg', 't130.jpeg', 't131.jpeg', 't132.jpeg', 't134.jpeg', 't135.jpeg', 't136.jpeg', 't137.jpeg', 't138.jpeg']
Network name: ./track1/code1_320_G_epoch9000_bs4.pth; The 0-th iteration
track2 0 t125.jpeg
track2 0 t126.jpeg
track2 0 t127.jpeg
track2 0 t128.jpeg
track2 0 t129.jpeg
track2 0 t130.jpeg
track2 0 t131.jpeg
track2 0 t132.jpeg
track2 0 t134.jpeg
track2 0 t135.jpeg
track2 0 t136.jpeg
track2 0 t137.jpeg
track2 0 t138.jpeg
Generator is loaded!
['t125.jpeg', 't126.jpeg', 't127.jpeg', 't128.jpeg', 't129.jpeg', 't130.jpeg', 't131.jpeg', 't132.jpeg', 't134.jpeg', 't135.jpeg', 't136.jpeg', 't137.jpeg', 't138.jpeg']
Network name: ./track1/code1_384_G_epoch6000_bs4.pth; The 1-th iteration
track2 1 t125.jpeg
track2 1 t126.jpeg
track2 1 t127.jpeg
track2 1 t128.jpeg
track2 1 t129.jpeg
track2 1 t130.jpeg
track2 1 t131.jpeg
track2 1 t132.jpeg
track2 1 t134.jpeg
track2 1 t135.jpeg
track2 1 t136.jpeg
track2 1 t137.jp

In [ ]:
in_path = './test/track1/0/output'
file_list_1 = []
for filename in os.listdir(in_path):
    split_list = []
    spectral = hdf5.loadmat(in_path + "/" + filename)
    #removing comments
    #img = spectral['cube']  # 31 channels (482,512,31)
    #print(img.shape)
    #print(img.dtype)
    split_path = in_path + '_split'
    if not os.path.exists(split_path):
        os.makedirs(split_path)
    for i in range(31):
        img = spectral['cube'][:, :, i]
        a = (img * 255).astype(np.uint8)
        new_dir = filename.replace('.mat', '')
        dir_path_s = split_path + '/' +new_dir
        if not os.path.exists(dir_path_s):
            os.makedirs(dir_path_s)
        new_file = dir_path_s + '/' + new_dir+ '_%d.jpg'%i
        #print(new_file)
        cv2.imwrite(new_file, a)
        #print(a)
        split_list.append(a)
    #print(len(spectral_list))
    file_list_1.append(split_list)
print(len(file_list_1))

1
